In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymorphy2
from pymorphy2.units.by_lookup import DictionaryAnalyzer
import re
import os
import json

%matplotlib inline

# Solver 9

In [2]:
morph = pymorphy2.MorphAnalyzer()
slovarnie_slova = pd.read_csv("../models/dictionaries/slovarnie_slova.txt", header=None).rename({0: "word"}, axis=1)


    
def solver_9(task, testing=False):
    def is_unverifiable(w):
        for w2 in slovarnie_slova.word:
            if re.match(re.sub(r"\.\.", ".", w), w2):
                return True
        return False

    def is_stressed(w, pos):
    #     stressed_w = accent.put_stress(w)
    #     if (stressed_w[pos+1] == "'") or ("'" not in stressed_w):
    #         return True
        return False

    def word_exists(w):
        analysis = morph.parse(w)
        if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and (analysis[0].methods_stack[0][1] == w):
            return True
        return False

    def possible_variants(w):
        amount = 0
        for candidate in "аоеиы":
            w_n = re.sub(r"\.\.", candidate, w)
            analysis = morph.parse(w_n)
            if (analysis[0].methods_stack[0][0].__class__.__name__ == "DictionaryAnalyzer") and (analysis[0].methods_stack[0][1] == w_n):
                amount += 1
        if amount == 0:
            amount = 1
        return amount

    def is_alternant(w):
        #зависящие от конечной согласной корня
        patterns_1 = [
            (r"[а-я]*р\.\.(ст|щ)[а-я]*", "а"),
            (r"[а-я]*р\.\.с[а-су-я]*", "о"),
            (r"[а-я]*л\.\.г[а-я]*", "а"),
            (r"[а-я]*л\.\.ж[а-я]*", "о"),
            (r"[а-я]*ск\.\.к[а-я]*", "а"),
            (r"[а-я]*ск\.\.ч[а-я]*", "о"),
        ]
        #зависящие от суффикса "а" после корня
        patterns_2 = [
            (r"[а-я]*(б|д|м|п|т)\.\.ра[а-я]*", "и"),
            (r"[а-я]*бл\.\.ста[а-я]*", "и"),
            (r"[а-я]*ж\.\.га[а-я]*", "и"),
            (r"[а-я]*ст\.\.ла[а-я]*", "и"),
            (r"[а-я]*ч\.\.та[а-я]*", "и"),
            (r"[а-я]*к\.\.са[а-я]*", "а"),
            (r"[а-я]*(б|д|м|п|т)\.\.р[б-я]*", "е"),
            (r"[а-я]*бл\.\.ст[б-я]*", "е"),
            (r"[а-я]*ж\.\.г[б-я]*", "е"),
            (r"[а-я]*ст\.\.л[б-я]*", "е"),
            (r"[а-я]*ч\.\.т[б-я]*", "е"),
            (r"[а-я]*к\.\.с[б-я]*", "о"),
        ]
        #зависящие от ударения (плов-плав хз почему тут, всегда пишется "а" кроме исключений)
        patterns_3a = [
            (r"[а-я]*з\.\.р[а-я]*", "оа"),
            (r"[а-я]*г\.\.р[а-я]*", "ао"),
            (r"[а-я]*тв\.\.р[а-нп-я]+", "ао"),
        ]
        patterns_3b = [
            (r"[а-я]*пл\.\.в[а-я]*", "оа"),
        ]
        #зависящие от лексического значения
        patterns_4 = [
            (r"[а-я]*м\.\.к[а-я]*", "оа"),
            (r"[а-я]*р\.\.вн[а-я]*", "оа"),
        ]

        exceptions = [
            "росток", "ростов", "ростислав", "ростовщик",
            "отрасль", "скачок", "скачу", "сочетать", "сочетание",
            "чета", "зоревать", "зорянка", "пловец", "пловчиха",
            "плывуны", "уровень", "ровесник", "равнина", "равняйсь", 
            "равнение ",
        ]
        w = w.lower()
        pos_space = re.search(r"\.", w).span()[0]
        for p in patterns_1:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 1, filled_w, pos_space, stressed
        for p in patterns_2:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 2, filled_w, pos_space, stressed
        for p in patterns_4:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 4, filled_w, pos_space, stressed
        for p in patterns_3a:
            if re.match(p[0], w):
                for q, p_i in enumerate(p[1]):
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stress_ind = is_stressed(filled_w, pos_space)
                        if (stress_ind) and (q == 0):
                            return True, 3, filled_w, pos_space, True
                        if (q == 1) and (not stress_ind):
                            return True, 3, filled_w, pos_space, False
        for p in patterns_3b:
            if re.match(p[0], w):
                for p_i in p[1]:
                    filled_w = re.sub(r"\.\.", p_i, w)
                    if word_exists(filled_w):
                        stressed = is_stressed(filled_w, pos_space)
                        return True, 3, filled_w, pos_space, stressed
        return False, None, None, pos_space, None

    words = np.array([re.split(r", ", t["text"]) for t in task["question"]["choices"]])
    #обрезаем скобки
    words = [[re.sub("\([а-я ]+\)", "", t2).strip() for t2 in t1] for t1 in words]
    #в зависимости от числа слов в каждом варианте, мы ожидаем разное число верных ответов
    num_answers = 2
    if len(words[0]) == 1:
        num_answers = 1
    #определяем какой тип нужно искать
    if "чередующ" in task["text"]:
        task_type = 0
    elif "непровер" in task["text"]:
        task_type = 1
    else:
        task_type = 2
    alt_labels = [[is_alternant(t2) for t2 in t1] for t1 in words]
    unver_labels = [[is_unverifiable(t2) for t2 in t1] for t1 in words]
    possible_ways = [[possible_variants(t2) for t2 in t1] for t1 in words]
    scores = np.zeros((len(words), len(words[0]), 3))
    for i in range(scores.shape[0]):
        for j in range(scores.shape[1]):
            scores[i, j, 0] = 0
            scores[i, j, 1] = unver_labels[i][j]
            if alt_labels[i][j][0]:
                scores[i, j, 0] = alt_labels[i][j][0]
            scores[i, j, 2] = 1 - scores[i, j, 0] - 10 * scores[i, j, 1] * (possible_ways[i][j]-1)
    if testing: print(scores)
    agg_scores = scores.mean(axis=1)
    if testing: print(agg_scores)
    agg_scores = agg_scores[:, task_type]
    if testing: print(agg_scores)
    max_score = agg_scores.max()
    second_value = agg_scores[agg_scores.argsort()[-2]]
    answer_numbers = np.arange(len(agg_scores))[agg_scores==max_score]
    if (len(answer_numbers) < 2) and (second_value > 0):
        answer_numbers = np.concatenate([answer_numbers,
                                         np.arange(len(agg_scores))[agg_scores==second_value]])
#     answer_numbers = agg_scores.argsort()[2:]
    answer_numbers += 1
    answer_numbers = [str(t) for t in answer_numbers]
    return answer_numbers

In [3]:
with open("/Users/edgy/Downloads/task_9/T7008.json", "r") as f:
    js = json.load(f)

In [4]:
js

{'tasks': [{'id': '9',
   'text': 'Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.',
   'meta': {'language': 'ru', 'source': 'yandex_test'},
   'attachments': [],
   'solution': {'correct_variants': [['1', '5'], ['5', '1']]},
   'score': 1,
   'question': {'type': 'multiple_choice',
    'min_choices': 1,
    'choices': [{'id': '1',
      'text': 'неприм..римость, ч..столюбие, раст..нуться'},
     {'id': '2', 'text': 'л..гичный, шт..мповать, вым..ршие (животные)'},
     {'id': '3', 'text': 'т..жёлый, разв..вающиеся (флаги), б..тонировать'},
     {'id': '4',
      'text': '(лучший) зап..вала (в хоре), пл..чистый, заж..гание'},
     {'id': '5', 'text': 'сб..жавший, выст..вка, пок..титься'}]}}]}

In [5]:
solver_9(js["tasks"][0], testing=True)

[[[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]]
[[0.         0.         1.        ]
 [0.33333333 0.         0.66666667]
 [0.         0.         1.        ]
 [0.33333333 0.         0.66666667]
 [0.         0.         1.        ]]
[1.         0.66666667 1.         0.66666667 1.        ]


['1', '3', '5']

In [6]:
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_9/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
scores_alt = []
scores_ver = []
scores_unver = []
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_9/{path_t}", "r") as f:
        js = json.load(f)
    if "correct" in js['tasks'][0]['solution']:
        true_ans = set(js['tasks'][0]['solution']['correct'])
    else: 
        true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])
    answer = set(solver_9(js['tasks'][0]))
    last_score = len(answer.intersection(true_ans)) / len(answer.union(true_ans))
    if "череду" in js['tasks'][0]["text"]:
        scores_alt.append(last_score)
    if "непровер" in js['tasks'][0]["text"]:
        scores_unver.append(last_score)
    else:
        scores_ver.append(last_score)
    if last_score < 0.6:
        print(f"Task {path_t.split('.')[0]}")
        print(js["tasks"][0]["text"])
        print(f"True answer: {true_ans}")
        print(f"Predicted answer: {answer}")

Task T5432
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'2', '5'}
Predicted answer: {'2', '3'}
Task T3749
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная проверяемая гласная корня.
True answer: {'2', '4', '3'}
Predicted answer: {'1', '3'}
Task T7013
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная непроверяемая гласная корня.
True answer: {'5', '3'}
Predicted answer: {'2', '1'}
Task T8470
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная непроверяемая гласная корня.
True answer: {'4', '1'}
Predicted answer: {'1', '3'}
Task T7005
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная чередующаяся гласная корня.
True answer: {'2', '5', '3'}
Predicted answer: {'2', '4', '3'}
Task T7639
Укажите варианты ответов, в которых во всех словах одного ряда пропущена безударная чередующая

In [7]:
print("Чередующаяся: ", np.mean(scores_alt))
print("Проверяемая: ", np.mean(scores_ver))
print("Непроверяемая: ", np.mean(scores_unver))
print("Total: ", np.mean(scores_unver+scores_alt+scores_ver))

Чередующаяся:  0.5340277777777778
Проверяемая:  0.5306306306306307
Непроверяемая:  0.6187499999999999
Total:  0.5499999999999999


In [8]:
paths_to_tasks = os.listdir("/Users/edgy/Downloads/task_9/")
paths_to_tasks = [t for t in paths_to_tasks if t.startswith("T")]
scores_alt = []
scores_ver = []
scores_unver = []
for path_t in paths_to_tasks:
    with open(f"/Users/edgy/Downloads/task_9/{path_t}", "r") as f:
        js = json.load(f)
    if "correct" in js['tasks'][0]['solution']:
        true_ans = set(js['tasks'][0]['solution']['correct'])
    else: 
        true_ans = set(js['tasks'][0]['solution']['correct_variants'][0])
    answer = {"1", "2", "3", "4", "5"}
    last_score = len(answer.intersection(true_ans)) / len(answer.union(true_ans))
    if "череду" in js['tasks'][0]["text"]:
        scores_alt.append(last_score)
    if "непровер" in js['tasks'][0]["text"]:
        scores_unver.append(last_score)
    else:
        scores_ver.append(last_score)
        
print("Чередующаяся: ", np.mean(scores_alt))
print("Проверяемая: ", np.mean(scores_ver))
print("Непроверяемая: ", np.mean(scores_unver))
print("Total: ", np.mean(scores_unver+scores_alt+scores_ver))

Чередующаяся:  0.4416666666666667
Проверяемая:  0.4540540540540541
Непроверяемая:  0.41250000000000003
Total:  0.44155844155844154


In [531]:
np.mean(scores_a)

0.44150943396226416